In [2]:
proj_id = 'cmat-315920'
root_path = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}


refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

for level in ['cntyvtd']:#'tabblock', 'cntyvtd', 'bg', 'tract', 'cnty']:
    g = Gerry(abbr='TX', level=level, agg_shapes=True,
              election_filters=(
                "office='President' and race='general'",
                "office='USSen' and race='general'",
#                 "office like 'USRep%' and race='general'",
              ),
              refresh_tbl=refresh_tbl, refresh_all=refresh_all
             )
#     g.get_data()
    g.MCMC(steps=4)
self = g

NameError: name 'cmat' is not defined